In [23]:
%set_env OPENAI_API_KEY=

env: OPENAI_API_KEY=


In [24]:
from typing import List
from pydantic import BaseModel, Field

In [2]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code:str = Field(description="airport code to get the weather for")

In [3]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [4]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [5]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'type': 'object',
  'properties': {'airport_code': {'description': 'airport code to get the weather for',
    'type': 'string'}},
  'required': ['airport_code']}}

In [6]:
from langchain.chat_models import ChatOpenAI

In [8]:
model = ChatOpenAI()

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
model.invoke("what is the weather in SF today?",functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 71, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-0943bd0c-7acf-4753-83de-7480b99a07b1-0')

In [10]:
#OR 

In [14]:
model_with_bind = model.bind(functions=[weather_function])

In [15]:
model_with_bind.invoke("what is the weather in SF today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 71, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-6fca00c0-55bf-449b-bf1f-5f7500a2e911-0')

In [16]:
from langchain.prompts import ChatPromptTemplate

In [17]:
prompt =  ChatPromptTemplate.from_messages(
    [
        ("system","you are an helpful assistant"),
        ("user","{input}")
    ]
)

chain = prompt | model

In [18]:
chain.invoke({"input":"what is the weather in sanfransisco"})

AIMessage(content="I'm sorry, but I do not have real-time information. You can check the weather in San Francisco by using a weather website or app such as Weather.com or AccuWeather.", response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 25, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-9ac9f478-be76-42ac-9ac3-28ce467fea73-0')

In [19]:
chain2 = prompt | model_with_bind

In [20]:
chain2.invoke({"input":"what is the weather in sanfransisco"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 78, 'total_tokens': 95}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-c09f8688-120a-48b5-8e0e-cc5a695c055b-0')